In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('fede'))
if module_path not in sys.path:
    sys.path.append(module_path)
# import pandas as pd
from client import Client
from supported_modles import Supported_modles
import utils

from global_model import GlobalModel
from supported_modles import Supported_modles
import utils
import numpy as np

In [2]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [3]:
selected_model = Supported_modles.NN_classifier

In [5]:
clients, test_x, test_y = utils.set_data_mock(selected_model, True, False)

Creating node1.
Creating node2.
Creating node3.
Creating node4.


In [60]:
f1_before = []
for client in clients:
    print(client.x.shape)
    client.init_empty_model(0.1)
    try:
        client.train_model(epochs=50)
    except ValueError:
        score = 0
        print(client.name)
        f1_before.append(score)
        continue 
    try: 
        score = client.test_model_f1(y_test=test_y, X_test=test_x)
    except IndexError:
        score = 0
    f1_before.append(score)
    print(score)

(100000, 78)
0.7446709708345192
(100000, 78)
0.7420468113642683
(100000, 78)
0.6660716835641869
(100000, 78)
0.0
(100000, 78)
0.560557745251441


In [61]:
f1_before

[0.7446709708345192,
 0.7420468113642683,
 0.6660716835641869,
 0.0,
 0.560557745251441]

In [62]:
clients.remove(clients[3])

In [63]:
global_model = GlobalModel(selected_model)

In [64]:

NUMBER_OF_CLIENTS = len(clients)
applicable_clients = []
start = True

print(f'Starting new round!')
f1s = np.empty((0,NUMBER_OF_CLIENTS), float)
data_weights = arr = np.empty((0,NUMBER_OF_CLIENTS), float)

for client in clients:
    print(client.name)
    if start:
        print(f'Initializing Global Model')
        global_model.model = client.model
        start = False

    applicable_clients.append(client) 
    data_weights = np.append(data_weights, client.x.shape[0])
    
round_weights = data_weights/sum(data_weights)
print(round_weights)

global_model.update_global_model(applicable_clients,round_weights,selected_model)

# for client in clients:
#     print(client.name)
#     f1_fede = global_model.f1_score(client.x_test,client.y_test)
#     f1_local = client.f1

#     print(f'{f1_fede}:{f1_local}')

#     if f1_fede > f1_local:
#         print("f1 fede was better")
#         client.f1 = f1_fede
#         # local was better set to local values
#         client.model = global_model.model
            
# print(f'++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

Starting new round!
node1
Initializing Global Model
node2
node3
node5
[0.25 0.25 0.25 0.25]


In [65]:
f1_before

[0.7446709708345192,
 0.7420468113642683,
 0.6660716835641869,
 0.0,
 0.560557745251441]

In [66]:
global_model.f1_score(test_x,test_y)

0.5658833908092068